# SDE
## *Simulating independent trajectories of a vector SDE*

------------

Demain

i want to create a function to simulate muttidimensional diffusion processes: a :maruyama scheme but I want to simulate in vectorized way R independent realization of the same process, the diffusion should be in dimension n and the wiener process in dimension d. The diffusion coefficient is f(t,X) and the diffusion coefficient is g(t,X), they should be given by the user. 

----------



$\newcommand{\rmd}{\textrm{d}}$
$\newcommand{\R}{\mathbb{R}}$
Consider a $n$-dimensional SDE:
$$
   dX_t = f(t,X_t)\,\rmd t + g(t,X_t)\,\rmd W_t\,,\quad t\in[0,T]
$$
where
- $X_t$ (resp. $W_t$) is of dimension $n$ (resp. $d$) 
- $W_t$ is a standard Brownian motion independent of $X_0$
- $f:\R\times \R^n\to \R^n$,  $g:\R\times \R^n\to \R^{n\times d}$

Indices are:
$$
X_t = [X_{t,i}]_{i=1:n}
\quad W_t = [W_{t,j}]_{j=1:d}
\quad f = [f_i]_{i=1:n}
\quad g=[g_{i,j}]_{i=1:n,j:1:d}
$$
I like also to simply write:
$$
X_t = X_{t,1:n}
\quad W_t = W_{t,1:d}
\quad f = f_{1:n}
\quad g = g_{1:n,1:d}
$$


> **Goal:** *generate $K$ independent trajectories $[0,T]\ni t \to X_t$*.

## Times discretization

We use an Euler-Maruyama scheme. Let $K$ given and:
$$
t_k := k\,\delta\,,\quad k=0:K
$$
where
$$
 \delta := \frac{T}{K}
$$
the approximation $\tilde X_k$ of $X_{t_k}$ is given by 
$$
  \tilde X_{k+1} = \tilde X_k + f(t_k,\tilde X_k)\,\delta + g(t_k,\tilde X_k)\,\sqrt{\delta}\,w_k
$$
where $w_k$ is i.i.d. $N(0,I_{d\times d})$ so that $\sqrt{\delta}\,w_k$ is an approximation of $W_{t_{k+1}}-W_{t_{k}}$. Here we suppose that $X_0=\tilde X_0=x$ given.


# Data structure

Simulate `R` independent trajectories of a  `d`-dimensional SDE `X` with drift coefficeint `f` and diffusion coefficient `g`. 

 |   **dimensions**  |                        |                                                    |
 | ----| -----------------------| ---------------------------------------------------| 
 | `n` |  state space dimension |  implicitly defined the diffusion coefficient `f`  | 
 | `d` |  noise dimension       |  implicitly defined the diffusion coefficient `g`  | 

 | **indices**      |         |
 |-------------------------|---------|
 | `i=1:n` | state dimension index |
 | `j=1:d` | noise dimension index   | 
 |   `r=1:R` |realization index      | 
 |   `k=0:N` |time index              `t_k = k * dt = i * T/N`| 

  | **output structures**   |               |               |
  |----------|-----------------------------------|---------------|
  | `t_vals` | array of time points of length    | `N+1`         |
  | `X_vals` | array of simulated paths of shape | `(R, N+1, n)` | 




```
      f: Function f(t, X) -> ndarray, drift term of SDE.
        g: Function g(t, X) -> ndarray of shape (n, d), diffusion term of SDE.
        X0: Initial condition, ndarray of shape (n,).
        T: Final time.
        N: Number of time steps (N+1: number of time instants)
        R: Number of independent realizations.

    X_vals: Array of shape (R, N+1, n) 
    X:      Array of shape (R, N+1, n) 

        drift = f(t, X)  # Shape (R, n)
        # Einstein summation convention :
        #    summation along j-index
        #    to get a Shape (R, n) from (R, d, n)*(R, n)
        diffusion = np.einsum('rji,rj->ri', g(t, X), dW[:, i, :])  # Shape (R, n)

        X_vals <- X + drift * dt + diffusion  # Update for all realizations
```


        

# Vectorized algorithm 

```
    n      = len(X0)                 # Dimension of X
    dt     = T / N                   # Time step size
    t_vals = np.linspace(0, T, N+1)  # Time points    
    X_vals = np.zeros((R, N+1, n))   # memory allocation
    X_vals[:, 0, :] = X0             # assign initial condition to all realizations

    dW = np.random.normal(0, np.sqrt(dt), size=(R, N, n))  # Brownian increments Shape (R, N, n)
                             >>>>>>>>>> should be (R, N, d)
                             >>>>>>>>>> TO BE FIXED

    for i in range(N):        # loop on time index (vectorized on realizations)
        t = t_vals[i]         # current time
        X = X_vals[:, i, :]   # current state (all realizations)
        drift     = f(t, X) 
        diffusion = np.einsum('rji,rj->ri', g(t, X), dW[:, i, :])
        X_vals[:, i+1, :] = X + drift * dt + diffusion # state update 
```

# The (vectorized) routine

In [1]:
import numpy as np

def sde_vectorized(f, g, X0, T, N, R):
    """
    Vectorized simulation of an SDE using the Euler-Maruyama scheme.

    Parameters:
        f: Function f(t, X) -> ndarray, drift term of SDE.
        g: Function g(t, X) -> ndarray of shape (n, d), diffusion term of SDE.
        X0: Initial condition, ndarray of shape (n,).
        T: Final time.
        N: Number of time steps (N+1: number of time instants)
        R: Number of independent realizations.

    Understand:
        n state space dimension given by n = len(X0)         i=1:n
        d noise dimension (implicitly defined ????)          j=1:d
        r realization index                                  r=1:R
        k time index, t_k = k * dt = i * T/N for k=0:N
        i dimension index, i=1:n

    Returns:
        t_vals: Array of time points.
        X_vals: Array of shape (R, N+1, n), simulated paths of X_t.
    """
    # --------------------------------------------------------------------------------
    # --- some tests about dimension coherence ---------------------------------------
    # --------------------------------------------------------------------------------

    # X0 will be of dimension (n,R)
    # n the dimension of the diffusion process and R
    # the number of independent realizations

    # Ensure X0 is a 2D array
    if not isinstance(X0, np.ndarray) or X0.ndim != 2:
        raise ValueError("X0 must be a 2D NumPy array of shape (n, R).")

    n, R = X0.shape  # Deduce n and R from X0

    # Dimensions
    n = len(X0)  # Dimension of X
    dt = T / N  # Time step size
    t_vals = np.linspace(0, T, N + 1)  # Time points

    # Pre-allocate array for results
    X_vals = np.zeros((R, N + 1, n))
    # Initialize paths
    X_vals[:, 0, :] = X0  # Set initial condition for all realizations

    # Check the dimensions of f and g before starting the simulation
    # Check the dimensions of f
    f_result = f(t[0], X[:, :, 0])  # Evaluate f at the first time step
    if f_result.shape != (n, R):
        raise ValueError(f"Function f(t, X) should return an array of shape (n, R), but got {f_result.shape}.")

    # Check the dimensions of g
    g_result = g(t[0], X[:, :, 0])  # Evaluate g at the first time step
    if g_result.shape != (n, d, R):  # Assuming d is the number of noise dimensions
        raise ValueError(f"Function g(t, X) should return an array of shape (n, d, R), but got {g_result.shape}.")

    # Generate Brownian increments for all realizations
    dW = np.random.normal(0, np.sqrt(dt), size=(R, N, n))  # Shape (R, N, n)

    # Vectorized iteration
    for i in range(N):
        current_t = t_vals[i]
        current_X = X_vals[:, i, :]  # Current state for all realizations
        drift = f(current_t, current_X)  # Shape (R, n)
        # Einstein summation convention :
        #    summation along j-index
        #    to get a Shape (R, n) from (R, d, n)*(R, n)
        diffusion = np.einsum("rji,rj->ri", g(current_t, current_X), dW[:, i, :])  # Shape (R, n)
        X_vals[:, i + 1, :] = current_X + drift * dt + diffusion  # Update for all realizations

    return t_vals, X_vals


## Example usage

In [2]:
# ___ system parameters __________________________________________________________________________

p_a, p_c, p_d, p_input = 0.08, 3, 1.8, 2.0   # parameter controlling the (x,y)-equation
p_x1         = -3.0                # one parameter of the z-component of the slow forcing
p_eps        = 0.01                # speed of the slow forcing
p_s1, p_alpha, p_z0, p_b0 = 0.01, -0.1, -1.7737, 1.5


def f(t, S):
    """
    Hindmarsh-Rose (HR) model
    """
    x, y, z, b = S
    f1 = p_c * (x-x**3/3-y+z+p_input)
    f2 = (x**2+p_d*x-b*y+p_a) / p_c
    f3 = p_eps * (-p_s1*(x-p_x1)-(b-p_b0))
    f4 = p_eps * (z-p_z0+p_alpha*x)
    return np.array([f1, f2, f3, f4])

def fff(t, S):
    x, y, z, b = S
    dxdt = p_c * (x-x**3/3-y+z+p_input)
    dydt = (x**2+p_d*x-b*y+p_a) / p_c
    dzdt = p_eps * (-p_s1*(x-p_x1)-(b-p_b0))
    dbdt = p_eps * (z-p_z0+p_alpha*x)
    return np.array([dxdt,dydt,dzdt,dbdt])



In [5]:
import numpy as np

#  the dimensions of X and W are given buy the funtcion f and g
#  the dimension of f depends on the dimension of X0

def f(t, X):
    return -X  # Example drift term

def g(t, X):
    return np.tile(np.eye(X.shape[1]), (X.shape[0], 1, 1))  # Example diffusion term (identity matrix)

# initial condition
#X0 = np.array([1.0, 0.0])  # Initial condition (2D system)
R = 5  # Number of repetitions
ini_cond = np.array([1.0, 0.0])
X0 = np.tile(ini_cond, (R, 1))


T = 1.0  # Final time
N = 100  # Number of time steps
R = 1000  # Number of realizations

t_vals, X_vals = sde_vectorized(f, g, X0, T, N, R)

ValueError: could not broadcast input array from shape (5,2) into shape (2,5)

# Graphics

In [ ]:
import matplotlib.pyplot as plt

# Enable LaTeX rendering
plt.rcParams['text.usetex'] = True

# Set global font size for labels
plt.rcParams['axes.labelsize'] = 20  # Font size for x and y labels
plt.rcParams['axes.titlesize'] = 20  # Font size for title
plt.rcParams['xtick.labelsize'] = 15  # Font size for x-axis ticks
plt.rcParams['ytick.labelsize'] = 15  # Font size for y-axis ticks

# Set the global font to Roman (serif) family
#plt.rcParams['font.family'] = 'serif'

# Set the font for LaTeX-style text
#plt.rcParams['serif'] = ['Times New Roman']  # Or any Roman font you prefer

plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = ["Times New Roman"]

## $R$ independent trajectories

In [ ]:
# Visualization of the paths (first dimension)
# --- we don't want to plot all the time steps
N_plots = 30              # number of times for plot
delta_plots = N//N_plots  # time steps between two plots

coordinate = 0

plt.figure(figsize=(10, 6))
for r in range(R):  # Plot first 10 realizations
    plt.plot(t_vals[::delta_plots], X_vals[r, ::delta_plots, coordinate], color='b', alpha=0.05)
plt.xlabel(r"time $t$")
plt.ylabel(r"$X_t$")
plt.title(rf"{R} simulated SDE paths (coordinnate {coordinate+1})")
plt.ylim(-3,3)
plt.show()

In [ ]:
# Example usage
def f(t, X):
    return -X  # Example drift term

def g(t, X):
    return np.eye(len(X))  # Example diffusion term (identity matrix)

# Parameters
X0 = np.array([2.0, 0.0])  # Initial condition (2D system)
T = 1.0  # Final time
N = 100  # Number of time steps
R = 1000  # Number of realizations

t_vals, X_vals = simulate_sde_euler_maruyama_vectorized(f, g, X0, T, N, R)

In [ ]:
aaaa = hindmarsh_rose_extended(1,np.array([1,2,3,4]))

In [ ]:
bbbb = aaaa.transpose()

In [ ]:
aaaa.shape

In [ ]:
bbbb.shape

In [ ]:
X0 = 1

In [ ]:
isinstance(X0, np.ndarray)

In [ ]:
len(X0.shape) == 1

In [ ]:
np.array([1,2,3,4])

In [ ]:
np.repmat()

In [ ]:
X0 = np.array([1])[:, np.newaxis]  # Shape (4, 1)
#X0 = np.tile(X0, 1)  # Shape (4, 10)

if X0.ndim == 1:
    n = X0.shape[0]
    R = 1  # Single realization applied to one trajectory
    print("cas 1")
elif X0.ndim == 2:
    n, R = X0.shape
    print("cas 2")
else:
    raise ValueError("X0 must be of shape (n,) or (n, R).")


In [ ]:
n = X0.shape[0]

In [ ]:
n

In [ ]:
X0 = X0[:, np.newaxis]

In [ ]:
X0

In [10]:
X0 = np.array([1,2,3,4])[:, np.newaxis]  # Shape (4, 1)
np.tile(X0, 10)  # Shape (4, 10)

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]])

In [16]:
R = 5  # Number of repetitions
ini_cond = np.array([1.0, 0.0])
np.tile(ini_cond, (R,1))


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])